In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=838f3d0828d95423d1385f2b40cd82d0a01524d9a513bd7fe6fbcc9f3740e697
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession  # Import the SparkSession class

spark = SparkSession.builder.appName("SparkSQLExample").getOrCreate()

In [4]:
#Full refresh: Load the entire dataset

df_sales =  spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("/content/sample_data/sales_data.csv")

#Apply transformations (if necessary)

df_transformed = df_sales.withColumn("total_sales", df_sales["quantity"] * df_sales["price"])

#Full refresh: Partition the data by 'date' and overwrite the existing data

output_path = "/content/sample_data/partitioned_data"

df_transformed.write.partitionBy("date").mode("overwrite").parquet(output_path)

#Verify partitioned data

partitioned_df = spark.read.parquet(output_path)

partitioned_df.show()

+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|transaction_id|customer_id| product|quantity|price|         updated_at|total_sales|      date|
+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|             1|        101|  Laptop|       1| 1000|2024-09-01 08:00:00|       1000|2024-09-01|
|             2|        102|   Phone|       2|  500|2024-09-01 09:00:00|       1000|2024-09-01|
|             5|        105|Keyboard|       1|   50|2024-09-03 12:00:00|         50|2024-09-03|
|             6|        106|   Mouse|       3|   30|2024-09-03 13:00:00|         90|2024-09-03|
|             3|        103|  Tablet|       1|  300|2024-09-02 10:00:00|        300|2024-09-02|
|             4|        104| Monitor|       2|  200|2024-09-02 11:00:00|        400|2024-09-02|
+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+

